In [ ]:
'''

python exp-3-synchronize-frames-of-recorded-video.py inputFile.mp4 tranFile.csv outputFile noFrames noCells tempDir/

'''

In [ ]:
# Include basics

import subprocess
import numpy as np
import cv2
import sys
import os
import time
import math


In [ ]:
#Extract frames
WIDTH = 1920
HEIGHT = 1080
OUT_WIDTH = WIDTH
OUT_HEIGHT = HEIGHT
OUTIMG = True



inputFileName = "../butterfly-recorded-20x20.mp4"#Video recorded by the phone
tranFileName = "../butterfly-code-timestamp-20x20.csv"#CSV timestamp,groundTruth generated by the PC
outputFileName = "../butterfly-20x20"#'arr0'--->frame recorded by camera, arr1------>label


if len(sys.argv)>=4:
    inputFileName=sys.argv[1]
    tranFileName=sys.argv[2]
    outputFileName=sys.argv[3]
    
print((OUT_HEIGHT, OUT_WIDTH))

SYN = 0
FINE = 0



In [ ]:
frameTS = []
output = subprocess.Popen("ffprobe -v error  -show_entries frame=pkt_pts_time -select_streams v -of csv=p=0 " + inputFileName, shell=True, stdout=subprocess.PIPE).stdout.read()
#print('FFPROBE output',output)
stamps = output.decode("utf-8").rstrip().split('\n')

for ts in stamps:
    frameTS.append(float(ts))
print("Timestamps loaded")

In [ ]:
NO_FRAMES=20000
NO_CELLS=400
TEMP_DIR="../tem/"

if len(sys.argv)>=7:
    NO_FRAMES=int(sys.argv[4])
    NO_CELLS=int(sys.argv[5])
    TEMP_DIR=sys.argv[6]


times = []
#pat55 = []
#pat33 = []
pat = []
print("Read tran file")
with open(tranFileName, 'r') as tranfile:
    line = tranfile.readline().rstrip()
    attrs = line.split(",")
    SYN = int(attrs[0])
    FIffprobeNE = float(attrs[1])
    for line in tranfile:
        attrs = line.rstrip().split(",")
        times.append(float(attrs[0]))
        pat.append(int(attrs[1]))

print("Num samples: " + str(len(times)))
violate = 0
imgset = []
tranindex = 1
index = 0
x_data = np.zeros((NO_FRAMES, 299, 299, 3),dtype=np.uint8)
y_label = np.zeros((NO_FRAMES,NO_CELLS), dtype=np.uint8)
video = cv2.VideoCapture(inputFileName)
#video.set(cv2.CAP_PROP_POS_FRAMES, 8990)K Sahiti, LM Kalanadhabhatta

while(video.isOpened()):
    ret, frame = video.read()
    if ret:
        ts = frameTS[index] - frameTS[SYN] + FINE
        dist0 = times[tranindex-1] - times[0]
        dist1 = times[tranindex] - times[0]
        patindex = tranindex - 10
        if ts >= dist0 and ts <= dist1:
            imgset.append(frame)
        elif ts > dist1:
            if len(imgset) <= 2:
                violate = violate + 1
            if OUTIMG == True:
                if (tranindex < 100) or (tranindex > NO_FRAMES-200):
                    cv2.imwrite(TEMP_DIR + str(patindex) + ".jpg", imgset[len(imgset)-1])
            if (patindex >= 0) and (patindex < NO_FRAMES):
                x_data[patindex,:,:] = cv2.resize(imgset[len(imgset)-1][HEIGHT-OUT_HEIGHT:HEIGHT, 0:OUT_WIDTH,:], (299,299),,interpolation = cv2.INTER_LINEAR)
                tem = pat[tranindex]
                for i in range(NO_CELLS):
                    y_label[patindex,i] = tem & 0x01
                    tem = tem >> 1
            tranindex = tranindex + 1
            imgset = [frame]
            print("Total: " + str(patindex) + ", Violate: " + str(violate))
            if tranindex >= len(times):
                break
        index = index + 1
        
        
        '''if index==NO_FRAMES:
            break'''
        
    else:
        break
print("Releasing video")
video.release()
cv2.destroyAllWindows()


print(patindex)

'''noDataPoints=x_data.shape[0]
noBatches=math.ceil(noDataPoints/100)


for b in range(noBatches):
    bb=b*100
    np.savez(outputFileName+'{:02}'.format(x)+'.npz', X=x_data[bb:min(bb+100,x_data.shape[0])], Y=y_label[bb:min(bb+100,x_data.shape[0])])'''

np.savez(outputFileName+'.npz', X=x_data, Y=y_label)

print("Finished")



In [ ]:
np.savez(outputFileName+'.npz', X=x_data[:7500], Y=y_label[:7500])
